<a href="https://colab.research.google.com/github/sumi1git/myproject/blob/main/FICC_conference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#load csv files
#ddf=pd.read_csv('/kaggle/input/mvsa-multiple-attcsv/mvsa_multiple_attr.csv')
df=pd.read_csv('/gdrive/MyDrive/sumana_nuig/simpson/simpson_attr_info.csv')
#df=pd.read_csv('/gdrive/MyDrive/sumana_nuig/mvsa_multiple/msva-multiple_jointlabel.csv')
#df=pd.read_csv('/gdrive/MyDrive/sumana_nuig/mvsa_multiple/mvsa_multiple_attr_clean.csv')
#df=pd.read_csv('/gdrive/MyDrive/sumana_nuig/mvsa/mvsa_attr_info1.csv')
df=df.dropna()

#df1=df1.iloc[:100:,]
df.shape
#df[:2]

(2830, 9)

In [ ]:
df[:2]

,img_path,text,object_names,object_score,att_name,att_score,joint_sentiment,class_att,bbox,class,class_text
0,img00021.jpg,white power movement new book connect movement...,"['eye', 'man', 'man', 'man', 'sign', 'sign', '...","[0.5621907711029053, 0.5024511814117432, 0.489...","['black', 'sitting', 'sitting', 'sitting', 'wh...","[0.4743715524673462, 0.18493443727493286, 0.15...",negative,"['eye black', 'man sitting', 'man sitting', 'm...","[[525.07763671875, 236.52786254882812, 609.050...",eye man man man sign sign ma...,white power movement new book connect movement...
1,img00024.jpg,lhate lama youral hat usiw anyth work jour tor...,"['floor', 'writing', 'writing', 'writing', 'wr...","[0.5603692531585693, 0.4662087559700012, 0.462...","['wooden', 'white', 'white', 'white', 'white',...","[0.304502934217453, 0.5460624694824219, 0.6122...",negative,"['floor wooden', 'writing white', 'writing whi...","[[0.0, 721.5868530273438, 416.1690368652344, 9...",floor writing writing writing writ...,lhate lama youral hat usiw anyth work jour tor...


In [ ]:
#For simpsion dataset
df['text'] = df['text'].apply(lambda x: x[:] if (len(x.split())<20) else (" ".join(x.split()[:20])))
import re
df["class"] = df["object_names"].apply(lambda x: re.sub("[\[{','}\]]", ' ', str(x)).strip())
df["text"]=df["text"].str.cat(df["text"])
#df["object_list"] = df["object_list"].apply(lambda x: re.sub("[\[{','}\]]", ' ', str(x)).strip())
df["class_text"]=df["text"].str.cat(df["class"])

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00


In [ ]:
pip install pandas pillow

In [ ]:
#image features and text features extarction using bert tokenization using pytorch
import torch
import torchvision.transforms as transforms
from transformers import BertTokenizer, BertModel
from sklearn.svm import SVC
import numpy as np
import tensorflow as tf

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Load VGG16 model
vgg16_model = torch.hub.load('pytorch/vision', 'vgg16', pretrained=True)
vgg16_model.eval()

# Preprocess image and text data
def preprocess_image(image_path):
    image_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    image_tensor = image_transform(image).unsqueeze(0)
    return image_tensor

def preprocess_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    return inputs

# Define the function to extract features from image using VGG16
def extract_image_features(image_tensor):
    with torch.no_grad():
        image_features = vgg16_model.features(image_tensor)
        image_features = vgg16_model.avgpool(image_features)
        image_features = torch.flatten(image_features, 1)
    return image_features



Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 79.7MB/s]


In [ ]:
#path='/gdrive/MyDrive/sumana_nuig/mvsa/data/'
#path='/gdrive/MyDrive/sumana_nuig/Dataset/'

# ... (Rest of the code remains the same)

# Load your labeled training data from .csv file
#train_data = pd.read_csv('/gdrive/MyDrive/sumana_nuig/mvsa/mvsa_attr_info1.csv')

# Process training data and extract features
text_features = []
image_features = []
labels = []
with tf.device('/device:GPU:0'): #when GPU is used
  for index, row in df.iterrows():
      text = row['class_text']
      #image_path = row['img_path']
      sentiment = row['joint_sentiment']

      text_inputs = preprocess_text(text)
      text_features.append((bert_model(**text_inputs).last_hidden_state.mean(dim=1)).detach().numpy())

      #image_tensor = preprocess_image(path+image_path)
      #image_features.append(extract_image_features(image_tensor).numpy())

      labels.append(sentiment)
text_features = np.vstack(text_features)
#image_features = np.vstack(image_features)
labels = np.array(labels)

# Combine text and image features into a multimodal feature matrix
#multimodal_features = np.hstack((text_features, image_features))

In [ ]:
text_features = np.vstack(text_features)
image_features = np.vstack(image_features)
labels = np.array(labels)

# Combine text and image features into a multimodal feature matrix
multimodal_features = np.hstack((text_features, image_features))

In [ ]:
##only for global image+TEMS-the combined text of class names and text
from sklearn.model_selection import train_test_split
from sklearn import svm
import tensorflow as tf
X_train, X_test, y_train, y_test = train_test_split(multimodal_features, labels, test_size=0.2)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
clf = svm.SVC(kernel='linear', C=1, probability=True)
with tf.device('/device:GPU:0'): #when GPU is used
  # train the classifier on the training data
  clf.fit(X_train, y_train)

  # evaluate the classifier on the test data
  #print(accuracy)
  # evaluate the classifier on the test data
  accuracy = clf.score(X_test, y_test)
  predicted = clf.predict(X_test)

  #ypredicted = np.argmax(predicted,axis=1)
  #y_predicted = np.where(predicted>0.5,1,0)
  print('Results of SVM')
  from sklearn.metrics import classification_report
  print(classification_report(y_test,predicted,digits=4))

(2264, 25856) (566, 25856) (2264,) (566,)
Results of SVM
              precision    recall  f1-score   support

    negative     0.4295    0.4085    0.4187       164
     neutral     0.5622    0.5381    0.5499       210
    positive     0.4833    0.5260    0.5037       192

    accuracy                         0.4965       566
   macro avg     0.4916    0.4909    0.4908       566
weighted avg     0.4970    0.4965    0.4962       566



In [ ]:
#only for TEMS-the combined text of class names and text
X_train, X_test, y_train, y_test = train_test_split(text_features, labels, test_size=0.2)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
clf = svm.SVC(kernel='linear', C=1, probability=True)
with tf.device('/device:GPU:0'): #when GPU is used
  # train the classifier on the training data
  clf.fit(X_train, y_train)

  # evaluate the classifier on the test data
  #print(accuracy)
  # evaluate the classifier on the test data
  accuracy = clf.score(X_test, y_test)
  predicted = clf.predict(X_test)

  #ypredicted = np.argmax(predicted,axis=1)
  #y_predicted = np.where(predicted>0.5,1,0)
  print('Results of SVM')
  from sklearn.metrics import classification_report
  print(classification_report(y_test,predicted,digits=4))

(2264, 768) (566, 768) (2264,) (566,)
Results of SVM
              precision    recall  f1-score   support

    negative     0.5363    0.6154    0.5731       156
     neutral     0.6216    0.5251    0.5693       219
    positive     0.5099    0.5393    0.5242       191

    accuracy                         0.5548       566
   macro avg     0.5559    0.5599    0.5555       566
weighted avg     0.5604    0.5548    0.5551       566



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
clf = svm.SVC(kernel='linear', C=1, probability=True)
# Train k-NN classifier on multimodal features for sentiment prediction
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed
knn_classifier.fit(multimodal_features, labels)
with tf.device('/device:GPU:0'): #when GPU is used
  # train the classifier on the training data
  knn_classifier.fit(X_train, y_train)

  # evaluate the classifier on the test data
  #print(accuracy)
  # evaluate the classifier on the test data
  accuracy = knn_classifier.score(X_test, y_test)
  predicted = knn_classifier.predict(X_test)

  #ypredicted = np.argmax(predicted,axis=1)
  #y_predicted = np.where(predicted>0.5,1,0)
  print('Results of K-NN')
  from sklearn.metrics import classification_report
  print(classification_report(y_test,predicted,digits=4))
# Train Random Forest classifier on multimodal features for sentiment prediction
rf_classifier = RandomForestClassifier(n_estimators=100)  # You can adjust the number of estimators (trees) as needed
rf_classifier.fit(multimodal_features, labels)
with tf.device('/device:GPU:0'): #when GPU is used
  # train the classifier on the training data
  rf_classifier.fit(X_train, y_train)

  # evaluate the classifier on the test data
  #print(accuracy)
  # evaluate the classifier on the test data
  accuracy = rf_classifier.score(X_test, y_test)
  predicted = rf_classifier.predict(X_test)

  #ypredicted = np.argmax(predicted,axis=1)
  #y_predicted = np.where(predicted>0.5,1,0)
  print('Results of RF')
  from sklearn.metrics import classification_report
  print(classification_report(y_test,predicted,digits=4))

Results of K-NN
              precision    recall  f1-score   support

    negative     0.3724    0.5705    0.4506       156
     neutral     0.6154    0.4018    0.4862       219
    positive     0.4511    0.4346    0.4427       191

    accuracy                         0.4594       566
   macro avg     0.4796    0.4690    0.4598       566
weighted avg     0.4930    0.4594    0.4617       566

Results of RF
              precision    recall  f1-score   support

    negative     0.5378    0.4103    0.4655       156
     neutral     0.5658    0.5890    0.5772       219
    positive     0.4932    0.5654    0.5268       191

    accuracy                         0.5318       566
   macro avg     0.5323    0.5216    0.5232       566
weighted avg     0.5336    0.5318    0.5294       566

